# Q1.

## Import library 

In [0]:
import pandas as pd
#!conda install -c conda-forge lxml --yes 

## Load data

In [3]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = data[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


### Pre-processing about 'Not assigned'

In [4]:
df = df[df.Borough != 'Not assigned']

for x in df[df.Neighbourhood == 'Not assigned'].index :
    df.loc[x].Neighbourhood = df.loc[x].Borough
    
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [5]:
df_ = df.groupby(by = ['Postcode','Borough'], sort=False, as_index=False).agg( ', '.join)
df_.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Explain:

This table is made by data from wiki's Canada_M(Toronto) Postal Codes 

About borough, Neighborhood,
If Neighborhood is a Not assigned then the neighborhood will be the same as the borough.

In [6]:
df_.shape

(103, 3)

# Q2.

## Load Geo data

In [0]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [8]:
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Extraction Latitude and Longitude positions

In [0]:
Lat_Long_df = pd.DataFrame(columns=['Latitude', 'Longitude'])

for x in df_.Postcode:
    Lat = geo_df[geo_df['Postal Code']== x].iloc[0,1]
    Long = geo_df[geo_df['Postal Code']== x].iloc[0,2]
    
    #data = pd.DataFrame(data = [[Lat, Long]], columns=['Latitude', 'Longitude'])    
    #Lat_Long_df = pd.DataFrame(Lat_Long_df.append(data))
    #Lat_Long_df = Lat_Long_df.reset_index(drop=True)
    
    Lat_Long_df = Lat_Long_df.append({'Latitude':Lat, 'Longitude':Long}, ignore_index=True)

In [10]:
Lat_Long_df.head(10)

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494
5,43.667856,-79.532242
6,43.806686,-79.194353
7,43.745906,-79.352188
8,43.706397,-79.309937
9,43.657162,-79.378937


### Merge dataframe

In [12]:
df_con = pd.concat([df_, Lat_Long_df], axis=1)
df_con.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### Draw on map

In [0]:
import folium # plotting library

latitude = df_con.Latitude.mean()
longitude = df_con.Longitude.mean()

#### Initialize map

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around Ecco

# add center circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

#### draw datas on map

In [0]:
for lat, lng, label in zip(df_con.Latitude, df_con.Longitude, df_con.Neighbourhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

In [16]:
venues_map